In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split

def load_data(file_path):
    data = pd.read_csv(file_path, encoding="latin-1")
    data = data[['Category', 'Message']]
    data.columns = ['label', 'message']
    data['label'] = data['label'].map({'ham': 0, 'spam': 1})
    return data

def split_data(data, test_size=0.2):
    X = data['message']
    y = data['label']
    return train_test_split(X, y, test_size=test_size, random_state=42)

if __name__ == "__main__":
    data = load_data("email.csv")
    X_train, X_test, y_train, y_test = split_data(data)
    print(f"Training samples: {len(X_train)}, Test samples: {len(X_test)}")

Training samples: 4458, Test samples: 1115


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import joblib
import os


def load_data(file_path):
    data = pd.read_csv(file_path, encoding="latin-1")
    expected_columns = ['Category', 'Message']
    if not all(col in data.columns for col in expected_columns):
        raise ValueError(f"The CSV file must contain columns: {expected_columns}. Found columns: {data.columns}")
    data = data[['Category', 'Message']]
    data.columns = ['label', 'message']
    data['label'] = data['label'].map({'ham': 0, 'spam': 1})
    return data

def split_data(data, test_size=0.2):
    X = data['message']
    y = data['label']
    return train_test_split(X, y, test_size=test_size, random_state=42)

def train_model(X_train, y_train):
    X_train = X_train[y_train.notna()]
    y_train = y_train[y_train.notna()]

    vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
    X_train_vec = vectorizer.fit_transform(X_train)
    model = MultinomialNB()
    model.fit(X_train_vec, y_train)
    return model, vectorizer

if __name__ == "__main__":
    data = load_data("email.csv")
    X_train, X_test, y_train, y_test = split_data(data)

    model, vectorizer = train_model(X_train, y_train)

    os.makedirs("model", exist_ok=True)


    joblib.dump(model, "model/spam_classifier.pkl")
    joblib.dump(vectorizer, "model/vectorizer.pkl")


    X_test_vec = vectorizer.transform(X_test)
    y_pred = model.predict(X_test_vec)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))

Accuracy: 0.97847533632287
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99       958
         1.0       1.00      0.85      0.92       157

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [18]:
import joblib

def predict_email(text):
    model = joblib.load("model/spam_classifier.pkl")
    vectorizer = joblib.load("model/vectorizer.pkl")
    text_vec = vectorizer.transform([text])
    prediction = model.predict(text_vec)
    return "Spam" if prediction[0] == 1 else "Ham"

if __name__ == "__main__":
    email = input("Enter an email message: ")
    print("Prediction:", predict_email(email))


Enter an email message: Get a free iPhone today! Limited stock available. Click here to claim: http://getfreeiphone.com
Prediction: Spam


In [15]:
import joblib

def predict_email(text):
    model = joblib.load("model/spam_classifier.pkl")
    vectorizer = joblib.load("model/vectorizer.pkl")
    text_vec = vectorizer.transform([text])
    prediction = model.predict(text_vec)
    return "Spam" if prediction[0] == 1 else "Ham"

if __name__ == "__main__":
    email = input("Enter an email message: ")
    print("Prediction:", predict_email(email))


Enter an email message: Hi John, I hope you’re doing well. Let’s meet for coffee tomorrow at 4 PM. Let me know if that works.
Prediction: Ham
